### Импортируем пакеты

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn.linear_model as lm

### Читаем входные файлы с данными

In [2]:
transactions = pd.read_csv('transactions.csv')

### Берём расходные транзакции. Добавляем признак day

In [3]:
train_transactions = transactions[transactions.amount < 0].copy()
train_transactions['day'] = train_transactions.tr_datetime.apply(lambda dt: dt.split()[0]).astype(int)

### Формируем тестовую выборку - последний месяц

In [4]:
test_transactions = pd.DataFrame(columns=train_transactions.mcc_code.unique(), 
                                 index=np.arange(1, 31) + train_transactions.day.max())
test_transactions = test_transactions.unstack().reset_index().dropna(axis=1)
test_transactions.columns = ['mcc_code', 'day']

### Формируем таблицу, состоящий из уникальных mcc_code и day. Она поможет нам подготовить данные для обучения

In [5]:
train_grid = pd.DataFrame(columns=train_transactions.mcc_code.unique(), 
                          index=train_transactions.day.unique())
train_grid = train_grid.unstack().reset_index().dropna(axis=1)
train_grid.columns = ['mcc_code', 'day']

### Определяем отображение дней в настоящие номера месяцев и в соответсвующие дни месяца (оказывается, это данные 2014-2015 годов)

In [6]:
aug0 = {i : 8 for i in range(0, 31)}
sep0 = {i : 9 for i in range(31, 61)}
oct0 = {i : 10 for i in range(61, 92)}
nov0 = {i : 11 for i in range(92, 122)}
dec0 = {i : 12 for i in range(122, 153)}

jan1 = {i : 1 for i in range(153, 184)}
feb1 = {i : 2 for i in range(184, 212)}
mar1 = {i : 3 for i in range(212, 243)}
apr1 = {i : 4 for i in range(243, 273)}
may1 = {i : 5 for i in range(273, 304)}
jun1 = {i : 6 for i in range(304, 334)}
jul1 = {i : 7 for i in range(334, 365)}
aug1 = {i : 8 for i in range(365, 396)}
sep1 = {i : 9 for i in range(396, 426)}
oct1 = {i : 10 for i in range(426, 457)}
nov1 = {i : 11 for i in range(457, 487)}

months = {}
for d in aug0, sep0, oct0, nov0, dec0, jan1, feb1, mar1, apr1, may1, jun1, jul1, aug1, sep1, oct1, nov1:
    months.update(d)
    
    
daug0 = {i : i + 1 for i in range(0, 31)}
dsep0 = {i : i - 30 for i in range(31, 61)}
doct0 = {i : i - 60 for i in range(61, 92)}
dnov0 = {i : i - 91 for i in range(92, 122)}
ddec0 = {i : i - 121 for i in range(122, 153)}

djan1 = {i : i - 152 for i in range(153, 184)}
dfeb1 = {i : i - 183 for i in range(184, 212)}
dmar1 = {i : i - 211 for i in range(212, 243)}
dapr1 = {i : i - 242 for i in range(243, 273)}
dmay1 = {i : i - 272 for i in range(273, 304)}
djun1 = {i : i - 303 for i in range(304, 334)}
djul1 = {i : i - 333 for i in range(334, 365)}
daug1 = {i : i - 364 for i in range(365, 396)}
dsep1 = {i : i - 395 for i in range(396, 426)}
doct1 = {i : i - 425 for i in range(426, 457)}
dnov1 = {i : i - 456 for i in range(457, 487)}

days = {}
for d in daug0, dsep0, doct0, dnov0, ddec0, djan1, dfeb1, dmar1, dapr1, dmay1, djun1, djul1, daug1, dsep1, doct1, dnov1:
    days.update(d)
    

### Формируем новые временные признаки с истинными их значениями

In [7]:
for tr_table in [train_transactions, test_transactions, train_grid]:
    tr_table['week_num'] = (tr_table['day'][tr_table.day >= 153] + 4) // 7 - 21 
    tr_table['week_num'] = tr_table['week_num'].fillna((tr_table.day + 5) // 7 + 31)
    tr_table['week_day'] = (tr_table['day'] + 5) % 7
    tr_table['month_num'] = tr_table['day'].apply(lambda dt: months[dt])
    tr_table['month_day'] = tr_table['day'].apply(lambda dt: days[dt])

### Группируем все данные, относящиеся к конкретному дню и к конкретному mcc_code

In [8]:
train_transactions = \
    pd.merge(train_grid,
             train_transactions.groupby(['day', 'week_num', 'week_day', 'month_num', 'month_day', 'mcc_code'])[['amount']]\
                 .sum().reset_index(),
             how='left').fillna(0)

### В качестве новых признаков берем историю транзакций примерно в течение года

In [9]:
for day_shift in range(7,365,7):
    train_shift = train_transactions.copy()
    train_shift['day'] += day_shift
    train_shift['amount_day_{}'.format(day_shift)] = np.log(-train_shift['amount'] + 500)
    train_shift = train_shift[['day', 'mcc_code', 'amount_day_{}'.format(day_shift)]]

    train_transactions = pd.merge(train_transactions, train_shift, 
                                      on=['day', 'mcc_code'], how='left')
    test_transactions = pd.merge(test_transactions, train_shift, 
                                     on=['day', 'mcc_code'], how='left')

In [10]:
col = train_transactions.columns.difference(['month_day'])
train_transactions = train_transactions[col]

### В тестовых данных пропущенное значение (Nan) заменяем на среднее значение по строке

In [11]:
test_transactions.iloc[:,6:]=test_transactions.iloc[:,6:].where(pd.notnull(test_transactions.iloc[:,6:]), test_transactions.iloc[:,6:].mean(1), axis = 0)

### В тренировочных данных не будем добавлять дополнительный шум, поэтому любые Nan просто удаляем. Таким образом, остаются данные по последним ~50 дням

In [12]:
train_transactions = train_transactions.dropna().reset_index()
train_transactions = train_transactions.iloc[:,1:]

### Обучаемся на полученных данных

In [13]:
shift = 500
train = pd.get_dummies(train_transactions, columns=['mcc_code', 'week_day'])
test = pd.get_dummies(test_transactions, columns=['mcc_code', 'week_day'])
c = train.columns.difference(['amount'])

clf = lm.Lasso(alpha = 0.0001, max_iter=4000)
clf.fit(train[c], np.log(-train['amount'] + shift))

Lasso(alpha=0.0001, copy_X=True, fit_intercept=True, max_iter=4000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [53]:
clf.coef_[:52].mean()

0.017826905837742907

In [60]:
np.abs(clf.coef_[:52]).sum() / 52

0.019191615265583075

In [59]:
1.0 / 52

0.019230769230769232

In [44]:
train[c].columns[[i for i, it in enumerate(clf.coef_) if it == 0]]

Index([u'day', u'mcc_code_5074', u'mcc_code_5441', u'mcc_code_5631',
       u'mcc_code_5722', u'mcc_code_5735', u'mcc_code_6051', u'mcc_code_8220',
       u'week_day_5'],
      dtype='object')

### Предсказываем объём трат для тестовых данных и создаём файл с ответом

In [14]:
test_transactions['volume'] = np.e ** clf.predict(test[c]) - shift
test_transactions[['mcc_code', 'day', 'volume']].to_csv('baseline_b.csv', index=False)